In [ ]:
#Import library
%matplotlib qt
import numpy as np
import staticCreepModule as creeplib
import matplotlib.pyplot as plt
import pandas as pd
import datetime 

# Input parameters

In [ ]:
#Configurations of how to process data
setOfSpecimens=1
deriveLVDT=True
timeInspection=False

# Initial functions

Get the path of the test results

In [ ]:
# Get path of the folder containing the test results
from tkinter.filedialog import askopenfilename, askdirectory
from tkinter import Tk
import os
root = Tk().withdraw()
path = askdirectory()
Tk().destroy()

General data

In [ ]:
'''
GENERAL INFO ABOUT THE TEST
----------------------------------------------------------------------------
SPECIMEN    BATCH   SYSTEM          LVDTS       BASE_LENGTH(MM)     APPLIED_STRESS(MPA)
----------------------------------------------------------------------------
SP1                 inegi        5,4,7       65.04667        8.47083
SP2                 inegi        9,8,6       57.63667        9.56737
SP3                 inegi        10,11,3       56.47667        9.66956
SP4                 inegi        1,2,0       56.50667           8.59628
----------------------------------------------------------------------------
'''
#Instantiate test specimens data
SP1=creeplib.testSpecimen("SP1",'inegi',[6,5,8],65.04667, 8.47083, ["SP1-LVDT_2A","SP1-LVDT_1A","SP1-LVDT_4A"], [(1,0,0),(1,0.40,0.40),(1,0.70,0.70)],['-','--','-.'])
SP2=creeplib.testSpecimen("SP2",'inegi',[10,9,7],57.63667, 9.56737,  ["SP2-LVDT_5A","SP2-LVDT_5","SP2-LVDT_3A"],[(0,0,0),(0.2,0.20,0.20),(0.70,0.70,0.70)],['-','--','-.'])
SP3=creeplib.testSpecimen("SP3",'inegi',[11,12,4],56.47667, 9.66956, ["SP3-LVDT_7","SP3-LVDT_9","SP3-LVDT_10"], [(0,1,1),(0.2,1,1),(0.4,1,1)],['-','--','-.'])
SP4=creeplib.testSpecimen("SP4",'inegi',[2,3,1],56.50667, 8.59628, ["SP4-LVDT_0","SP4-LVDT_1","SP4-HS_25"], [(1,1,1),(0.95,1,1),(0.90,1,1)],['-','--','-.'])
#Instasntiate load cell data
loadCellInterface=creeplib.loadCell("Interface-LC", 'inegi', 13, (1,0,0), '--')
#Instantiate loading switch data
loadingSwitch=creeplib.loadingSwitch()

In [ ]:
#Option 1:
if setOfSpecimens==0:
    experiment = creeplib.experiment([SP1,SP2,SP3,SP4],[loadCellInterface],loadingSwitch)
elif setOfSpecimens==1:
    experiment = creeplib.experiment([SP1,SP2,SP4],[loadCellInterface],loadingSwitch, numberOfActiveChannels=13)
elif setOfSpecimens==2:
    experiment = creeplib.experiment([SP2,SP4],[loadCellInterface],loadingSwitch, numberOfActiveChannels=13)
elif setOfSpecimens==3:
    experiment = creeplib.experiment([SP3,SP2,SP4],[loadCellInterface],loadingSwitch, numberOfActiveChannels=13)
elif setOfSpecimens==4:
    experiment = creeplib.experiment([SP1,SP2,SP3],[loadCellInterface],loadingSwitch, numberOfActiveChannels=13)


Handle release and slip corrections in LVDT and ring

In [ ]:
#Deal with sudden releases occured in the LVDTs, when they get unstucked
#For line number, subtract 25 from the line number in Excel - this means subtracting the entire header of the file, counting from the first line of acquired data
#Remeber that if, for a given specimen, more than 1 LVDT line needs correction per file, than they all should be stated in the same method call
#This framework is probably not prepared if more than 1 correction per file is needed. If such case happens, we need to reimplement some things

SP2.addReleaseCorrection('2023-07-17-12-42-S26.csv',["SP2-LVDT_5"],[547],[-0.16647228])
SP2.addReleaseCorrection('2023-07-24-20-02-S31.csv',["SP2-LVDT_5"],[233],[0.17222328])

SP2.addReleaseCorrection('2023-08-02-08-47-S36.csv',["SP2-LVDT_5"],[1391],[-0.16036344])
SP2.addReleaseCorrection('2023-10-02-18-05-S54.csv',["SP2-LVDT_5"],[1],[0.07261596])
SP2.addReleaseCorrection('2023-10-08-17-02-S59.csv',["SP2-LVDT_5"],[1],[-0.07261596])
SP4.addReleaseCorrection('2023-10-02-18-05-S54.csv',["SP4-HS_25"],[1],[0.0595])

SP4.addReleaseCorrection('2023-07-13-08-51-S23.csv',["SP4-HS_25"],[50],[0.0198])

### Read data

In this test, there occurred a power shortage between "2024-04-18 10:25" and "2024-04-21 00:00", which made the temperature fall and affected the creep tendency momentarily. This is not ideal but I think the results will not go to waste and we can use them simply skipping this period of test so the curves don't see a tooth in the middle of their trend

In [ ]:
experiment.readCreep_Batch(path, 'fileName')

# Set beginning of test data

## Description

To get the information required in the .setStartOfCreep methods below, use the Jupyter notebook timeStampAnalysis

## Time-stamp the experimental data

SP5 1009.862
SP2 218.832
SP3 857.628
SP4 392.78

In [ ]:

#Set beginning of test data
if setOfSpecimens==0:
    '''
    #These were the original times used
    SP1.setStartOfTest(datetime(year=2023, month=6, day=29, hour=11,minute=29, second=5))
    SP2.setStartOfTest(datetime(year=2023, month=6, day=29, hour=11,minute=32, second=34))
    SP3.setStartOfTest(datetime(year=2023, month=6, day=29, hour=11,minute=35, second=15))
    SP4.setStartOfTest(datetime(year=2023, month=6, day=29, hour=11,minute=38, second=11))
    '''
    '''
    #These were the times used to compare against loading switch
    SP1.setStartOfTest(datetime.datetime(year=2023, month=6, day=29, hour=11,minute=27, second=0))
    SP2.setStartOfTest(datetime.datetime(year=2023, month=6, day=29, hour=11,minute=27, second=0))
    SP3.setStartOfTest(datetime.datetime(year=2023, month=6, day=29, hour=11,minute=27, second=0))
    SP4.setStartOfTest(datetime.datetime(year=2023, month=6, day=29, hour=11,minute=38, second=0))
    '''
    #These are the final times used to plot (30 seconds after the full load identified from loading switch times)
    SP1.setStartOfTest(datetime.datetime(year=2023, month=6, day=29, hour=11,minute=28, second=25, microsecond=953700))
    SP2.setStartOfTest(datetime.datetime(year=2023, month=6, day=29, hour=11,minute=31, second=53, microsecond=520810))
    SP3.setStartOfTest(datetime.datetime(year=2023, month=6, day=29, hour=11,minute=34, second=35, microsecond=191000))
    SP4.setStartOfTest(datetime.datetime(year=2023, month=6, day=29, hour=11,minute=37, second=17, microsecond=773000))

    SP1.setStartOfCreep(datetime.datetime(year=2023, month=6, day=29, hour=11,minute=29, second=45, microsecond=953700))
    SP2.setStartOfCreep(datetime.datetime(year=2023, month=6, day=29, hour=11,minute=33, second=13, microsecond=520810))
    SP3.setStartOfCreep(datetime.datetime(year=2023, month=6, day=29, hour=11,minute=35, second=55, microsecond=191000))
    SP4.setStartOfCreep(datetime.datetime(year=2023, month=6, day=29, hour=11,minute=38, second=37, microsecond=773000))
elif setOfSpecimens==1:
    SP1.setStartOfTest(datetime.datetime(year=2023, month=6, day=29, hour=11,minute=28, second=25, microsecond=953700))
    SP2.setStartOfTest(datetime.datetime(year=2023, month=6, day=29, hour=11,minute=31, second=53, microsecond=520810))
    SP4.setStartOfTest(datetime.datetime(year=2023, month=6, day=29, hour=11,minute=37, second=17, microsecond=773000))

    SP1.setStartOfCreep(datetime.datetime(year=2023, month=6, day=29, hour=11,minute=29, second=45, microsecond=953700))
    SP2.setStartOfCreep(datetime.datetime(year=2023, month=6, day=29, hour=11,minute=33, second=13, microsecond=520810))
    SP4.setStartOfCreep(datetime.datetime(year=2023, month=6, day=29, hour=11,minute=38, second=37, microsecond=773000))
elif setOfSpecimens==2:
    SP2.setStartOfTest(datetime.datetime(year=2023, month=6, day=29, hour=11,minute=31, second=53, microsecond=520810))
    SP4.setStartOfTest(datetime.datetime(year=2023, month=6, day=29, hour=11,minute=37, second=17, microsecond=773000))

    SP2.setStartOfCreep(datetime.datetime(year=2023, month=6, day=29, hour=11,minute=33, second=13, microsecond=520810))
    SP4.setStartOfCreep(datetime.datetime(year=2023, month=6, day=29, hour=11,minute=38, second=37, microsecond=773000))
elif setOfSpecimens==3:
    SP2.setStartOfTest(datetime.datetime(year=2023, month=6, day=29, hour=11,minute=31, second=53, microsecond=520810))
    SP3.setStartOfTest(datetime.datetime(year=2023, month=6, day=29, hour=11,minute=34, second=35, microsecond=191000))
    SP4.setStartOfTest(datetime.datetime(year=2023, month=6, day=29, hour=11,minute=37, second=17, microsecond=773000))

    SP2.setStartOfCreep(datetime.datetime(year=2023, month=6, day=29, hour=11,minute=33, second=13, microsecond=520810))
    SP3.setStartOfCreep(datetime.datetime(year=2023, month=6, day=29, hour=11,minute=35, second=55, microsecond=191000))
    SP4.setStartOfCreep(datetime.datetime(year=2023, month=6, day=29, hour=11,minute=38, second=37, microsecond=773000))
elif setOfSpecimens==4:
    SP1.setStartOfTest(datetime.datetime(year=2023, month=6, day=29, hour=11,minute=28, second=25, microsecond=953700))
    SP2.setStartOfTest(datetime.datetime(year=2023, month=6, day=29, hour=11,minute=31, second=53, microsecond=520810))
    SP3.setStartOfTest(datetime.datetime(year=2023, month=6, day=29, hour=11,minute=34, second=35, microsecond=191000))

    SP1.setStartOfCreep(datetime.datetime(year=2023, month=6, day=29, hour=11,minute=29, second=45, microsecond=953700))
    SP2.setStartOfCreep(datetime.datetime(year=2023, month=6, day=29, hour=11,minute=33, second=13, microsecond=520810))
    SP3.setStartOfCreep(datetime.datetime(year=2023, month=6, day=29, hour=11,minute=35, second=55, microsecond=191000))

loadingSwitch.addOffset(datetime.datetime(year=2023, month=6, day=29, hour=11,minute=27, second=10, microsecond=559856),datetime.timedelta(seconds=56, microseconds=661484))


# Trustness correction

This correction is an idea to use only the LVDT lines which we have trust on their measurements. This may allow to savage data from specimens with bad LVDTs that would, otherwise, be thrown in the garbage. Maybe later spcimens can be pondered in the average results by their associated uncertainty (which would definetly allow for penalizing those specimens with fewer LVDTs).

The code below performs the thrustness analysis and store the results in a dataframe that gets exported to a xlsx in the end

Manual correction

In [ ]:
#SP3.displacementData['SP3-LVDT_9']=[0.57*(value1+value2) for value1, value2 in zip(SP3.displacementData['SP3-LVDT_7'],SP3.displacementData['SP3-LVDT_10'])]
#SP1.displacementData['SP1-LVDT_4A']=[0.2*(value1+value2) for value1, value2 in zip(SP1.displacementData['SP1-LVDT_1A'],SP1.displacementData['SP1-LVDT_2A'])]
SP3.displacementData['SP3-LVDT_9']=[np.mean([0.694*(value1),1.171*value2]) for value1, value2 in zip(SP3.displacementData['SP3-LVDT_7'],SP3.displacementData['SP3-LVDT_10'])]
SP1.displacementData['SP1-LVDT_4A']=[np.mean([0.162*(value1),0.157*value2]) for value1, value2 in zip(SP1.displacementData['SP1-LVDT_1A'],SP1.displacementData['SP1-LVDT_2A'])]

Make the corrections based on the trust analysis (this series does not have trust corrections)

In [ ]:
# Get path of the folder containing the test results

performCorrection = False

if performCorrection is True:
    from tkinter.filedialog import askopenfilename, askdirectory
    from tkinter import Tk
    import os
    import pandas as pd
    window = Tk()
    window.wm_attributes('-topmost', 1)
    window.withdraw()   # this supress the tk window
    correctionPathFile = askopenfilename(parent=window, title="Open file of loading switch")
    Tk().destroy()

    trustCorrectionParameters=pd.read_excel(correctionPathFile)
    untrustedLinesIndex=trustCorrectionParameters.loc[:, 'L1-trust?':'L3-trust?'].values.tolist()

    for j, specimenAnalysed in enumerate(experiment.testSpecimensList):
        #Get data series
        keywordTrustfulLines=[]
        keywordUntrustfulLines=[]
        trustfulLines=[]
        sumOfTrustfulLines=[0 for el in list(specimenAnalysed.displacementData.values())[0]]

        for i, series in enumerate(specimenAnalysed.displacementData):
            if untrustedLinesIndex[j][i] is True or untrustedLinesIndex[j][i]==1.0:
                #keywordTrustfulLines.append(series)
                #trustfulLines.append(specimenAnalysed.displacementData[series])
                for k, measurement in enumerate(specimenAnalysed.displacementData[series]):
                    sumOfTrustfulLines[k]=sumOfTrustfulLines[k]+measurement/trustCorrectionParameters.loc[j,'TrustSumFactor']
            elif untrustedLinesIndex[j][i] is False or untrustedLinesIndex[j][i]==0.0:
                keywordUntrustfulLines.append(series)
            else:
                continue #No corrections should be applied

        #Now correct the unstrustful lines:
        #First extract the string representation from the dataframe and transform it in a list of floats
        s=trustCorrectionParameters.loc[j,'UntrustFactors']
        elements = s.strip('[]').split(', ')
        if elements[0]=="None":
            continue
        # Convert each element to float
        untrustFactors=list(map(float, elements))
        for i, series in enumerate(keywordUntrustfulLines):
            specimenAnalysed.displacementData[series]=[value*untrustFactors[i] for value in sumOfTrustfulLines]

# Compute the information of creep

In [ ]:
experiment.computeCompliances()
experiment.computeSpecificCreep()
experiment.computeStatisticalMeasures()

Save data in csv files

Plot graphs

In [ ]:
experiment.pltAverageSpecificCreep(logScale = True, specimenResults=True)

In [ ]:
experiment.pltAverageCompliance(logScale = True, specimenResults=True)

In [ ]:
experiment.pltDisplacementData(logScale = True)

In [ ]:
experiment.pltSpecimenSpecificCreep(logScale = True)

In [ ]:
experiment.pltCoefficientOfVariationSpecificCreep()

In [ ]:
experiment.plotInterpolatedComplianceData(logScale=True)

In [ ]:
from tkinter.filedialog import asksaveasfile
from tkinter import Tk
import os
import pandas as pd
window = Tk()
window.wm_attributes('-topmost', 1)
window.withdraw()   # this supress the tk window
savePickleFilePath = asksaveasfile(parent=window, title="Open file of loading switch")
Tk().destroy()

experiment.saveExperimentObject(savePickleFilePath.name)

In [ ]:
import pickle

output = open(savePickleFilePath.name, 'wb')
# Pickle dictionary using protocol 0.
pickle.dump(experiment, output)
output.close()

Customized graphs

In [ ]:
#Piece of code used to see specimen and loading switch data
specimenAnalysed=SP4
fig,ax = plt.subplots()

for series in specimenAnalysed.displacementData:
    #deltaTime=SP4.timeData-SP4.timeData[0]
    #deltaTime=[time-SP2.timeData[0] for time in SP2.timeData]
    #shiftedTimeData=[(value-datetime.timedelta(seconds=specimenAnalysed.creepStartTime)-specimenAnalysed.timeData[0]).total_seconds() for value in specimenAnalysed.timeData]
    ax.plot(specimenAnalysed.plotTime, specimenAnalysed.displacementData[series], label=series)

#plt.xscale("log")
plt.legend()
plt.show()

In [ ]:
#Piece of code used to see specimen and loading switch data
specimenAnalysed=SP5
fig,ax = plt.subplots()
keywordTrustfulLines=[]
for series in specimenAnalysed.displacementData:
    keywordTrustfulLines.append(series)
    #deltaTime=SP4.timeData-SP4.timeData[0]
    #deltaTime=[time-SP2.timeData[0] for time in SP2.timeData]

zeroOneRatio=[]
for value1, value2 in zip(specimenAnalysed.displacementData[keywordTrustfulLines[0]],specimenAnalysed.displacementData[keywordTrustfulLines[1]]):
    try:
        value=value1/value2
    except:
        value=0
    zeroOneRatio.append(value)
zeroTwoRatio=[]
for value1, value2 in zip(specimenAnalysed.displacementData[keywordTrustfulLines[0]],specimenAnalysed.displacementData[keywordTrustfulLines[2]]):
    try:
        value=value1/value2
    except:
        value=0
    zeroTwoRatio.append(value)
oneTwoRatio=[]
for value1, value2 in zip(specimenAnalysed.displacementData[keywordTrustfulLines[1]],specimenAnalysed.displacementData[keywordTrustfulLines[2]]):
    try:
        value=value1/value2
    except:
        value=0
    oneTwoRatio.append(value)

ax.plot(specimenAnalysed.timeData, zeroOneRatio, label=f"{keywordTrustfulLines[0]}/{keywordTrustfulLines[1]}")
ax.plot(specimenAnalysed.timeData, zeroTwoRatio, label=f"{keywordTrustfulLines[0]}/{keywordTrustfulLines[2]}")
ax.plot(specimenAnalysed.timeData, oneTwoRatio, label=f"{keywordTrustfulLines[1]}/{keywordTrustfulLines[2]}")
plt.legend()
plt.show() 

In [ ]:
#Piece of code used to see specimen and loading switch data
specimenAnalysed=SP2
fig,ax = plt.subplots()

for series in specimenAnalysed.displacementData:
    keywordTrustfulLines.append(series)
    #deltaTime=SP4.timeData-SP4.timeData[0]
    #deltaTime=[time-SP2.timeData[0] for time in SP2.timeData]
    #shiftedTimeData=[(value-datetime.timedelta(seconds=specimenAnalysed.creepStartTime)-specimenAnalysed.timeData[0]).total_seconds() for value in specimenAnalysed.timeData]
    ax.plot(specimenAnalysed.plotTime, specimenAnalysed.displacementData[series], label=series)

#plt.xscale("log")
plt.legend()
plt.show()

In [ ]:
#Piece of code used to see specimen and loading switch data
specimenAnalysed=SP3
fig,ax = plt.subplots()

for series in specimenAnalysed.displacementData:
    keywordTrustfulLines.append(series)
    #deltaTime=SP4.timeData-SP4.timeData[0]
    #deltaTime=[time-SP2.timeData[0] for time in SP2.timeData]
    shiftedTimeData=[(value-datetime.timedelta(seconds=specimenAnalysed.creepStartTime)-specimenAnalysed.timeData[0]).total_seconds() for value in specimenAnalysed.timeData]
    ax.plot(shiftedTimeData, specimenAnalysed.displacementData[series], label=series)
    #ax.plot(specimenAnalysed.plotTime, specimenAnalysed.displacementData[series], label=series)

#plt.xscale("log")
plt.legend()
plt.show()


Try to correct specimen data by ensuring initial proportions from initial displacements remain constant during the creep test. For this, first the proportions between LVDT readings at the beginning of test are calculated (ex. LVDT1 represents 30% of the total sum of displacements). Then, trustyworth LVDT lines for each specimen are selected after visual inspection. The trustyworth lines will be used to estimate the total sum of displacement, and the untrustworthy lines will be substitute by their respective estimation based on the initial proportion of displacmeents.

In [ ]:
#Piece of code used to see specimen and loading switch data
specimenAnalysed=SP5
trustSumIndex=3
trustLineIndex=[0,1]

trustSumFactor=[0.710697149,0.83089024,0.250535332,0.787567894]
untrustFactor=[[0.289302851],[0.16910976],[0.2751606,0.474304069],[0.212432106]]


fig,ax = plt.subplots()
keywordTrustfulLines=[]
for series in specimenAnalysed.displacementData:
    keywordTrustfulLines.append(series)
    #deltaTime=SP4.timeData-SP4.timeData[0]
    #deltaTime=[time-SP2.timeData[0] for time in SP2.timeData]

if trustSumIndex == 2:
    #Then we are with SP4, which has only one trust line
    sumOfLines=[(value1)/trustSumFactor[trustSumIndex] for value1 in specimenAnalysed.displacementData[keywordTrustfulLines[trustLineIndex[0]]]]
else:
    #All the other specimens have 2 trust lines
    sumOfLines=[(value1+value2)/trustSumFactor[trustSumIndex] for value1, value2 in zip(specimenAnalysed.displacementData[keywordTrustfulLines[trustLineIndex[0]]],specimenAnalysed.displacementData[keywordTrustfulLines[trustLineIndex[1]]])]

j=0
for i in range(3):
    if i in trustLineIndex:
        ax.plot(specimenAnalysed.timeData, specimenAnalysed.displacementData[keywordTrustfulLines[i]], label=keywordTrustfulLines[i])
    else:
        print(j)
        ax.plot(specimenAnalysed.timeData, [value*untrustFactor[trustSumIndex][j] for value in sumOfLines], label=keywordTrustfulLines[i])
        j=j+1
plt.legend()
plt.show()

Save data in csv files

In [ ]:
# Get path of the folder containing the test results
from tkinter.filedialog import askopenfilename, askdirectory
from tkinter import Tk
import os
root = Tk().withdraw()
path = askdirectory()
Tk().destroy()
experiment.saveAverageDisplacementData(path+r'/averageDisplacement')
experiment.saveAverageCompliance(path+r'/averageCompliance')
experiment.saveAverageSpecificCreep(path+r'/specificCompliance')